### Scenario 1, Prompt 1 - Data (De)anonymization for enhanced privacy

Visit [webpage](https://pndang.com/GenAI_Capstone_ProjectOne/) for more details.

Test prompt: 1 - 
""" 
    Please translate the following to Vietnamese: 

    "Hi Robert! Please see below for your requested info

    full name: Phu Dang
    SSN: 123-45-1119
    email: pndang@ucsd.edu
    birthday: 04/11/2003, so I am 21 years old
    What's your nationality? I am a Vietnamese citizen
    What's your credit score? 775 FICO
    What's the loan amount requested? $1,000,000
    What's the loan purpose? Invest $500,000 in Sustana Cooling Partners as working capital and cash out the rest for my hi-performing restaurant business (Beers & Bastards, LLC). 

    Please ping me the best terms you could offer. Thanks!" 

"""

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

GPT_MODEL = 'gpt-4' 

# API Configuration
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
OpenAI.api_key = os.getenv('OPENAI_API_KEY') 

import warnings
warnings.filterwarnings("ignore")

#### Protect AI's LLM Guard first up!

In [3]:
from llm_guard import scan_output, scan_prompt 
from llm_guard.input_scanners import Anonymize
from llm_guard.output_scanners import Deanonymize
from llm_guard.vault import Vault  
# anonymized contents are stored in Vault (a storage mechanism) to later re-identify prompts

# EXTRA IMPORTS
from llm_guard.input_scanners import Regex
from llm_guard.input_scanners.regex import MatchType

In [4]:
# set up LLM Guard components

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
vault = Vault()
input_scanners = [Anonymize(vault)]
output_scanners = [Deanonymize(vault)]

2024-12-01 05:38:59 [debug    ] No entity types provided, using default default_entities=['CREDIT_CARD', 'CRYPTO', 'EMAIL_ADDRESS', 'IBAN_CODE', 'IP_ADDRESS', 'PERSON', 'PHONE_NUMBER', 'US_SSN', 'US_BANK_NUMBER', 'CREDIT_CARD_RE', 'UUID', 'EMAIL_ADDRESS_RE', 'US_SSN_RE']
2024-12-01 05:38:59 [debug    ] Initialized NER model          device=device(type='cpu') model=Model(path='Isotonic/deberta-v3-base_finetuned_ai4privacy_v2', subfolder='', revision='9ea992753ab2686be4a8f64605ccc7be197ad794', onnx_path='Isotonic/deberta-v3-base_finetuned_ai4privacy_v2', onnx_revision='9ea992753ab2686be4a8f64605ccc7be197ad794', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cpu'), 'aggregation_strategy': 'simple'}, tokenizer_kwargs={'model_input_names': ['input_ids', 'attention_mask']})
2024-12-01 05:38:59 [debug    ] Loaded regex pattern           group_name=CREDIT_CARD_RE
2024-12-01 05:38:59 [debug    ] Loaded regex pattern        

In [5]:
# test prompt

prompt = """Please translate the following to Vietnamese: 

    "Hi Robert! Please see below for your requested info

    full name: Phu Dang
    SSN: 123-45-1119
    email: pndang@ucsd.edu
    birthday: 04/11/2003, so I am 21 years old
    What's your nationality? I am a Vietnamese citizen
    What's your credit score? 775 FICO
    What's the loan amount requested? $1,000,000
    What's the loan purpose? Invest $500,000 in Sustana Cooling Partners as working capital and cash out the rest for my hi-performing restaurant business (Beers & Bastards, LLC). 

    Please ping me the best terms you could offer. Thanks!"
    """

In [6]:
# run prompt scanner pipeline

sanitized_prompt, results_valid, results_score = scan_prompt(input_scanners, prompt)
if any(results_valid.values()) is False:
    print(f"Prompt {prompt} is not valid, scores: {results_score}")
    exit(1)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


2024-12-01 05:39:03 [warning  ] Found unrecognized label, returning entity as is label=CREDITCARDCVV
2024-12-01 05:39:03 [warning  ] Found entity which is not supported by Presidio entity=DATE_OF_BIRTH
2024-12-01 05:39:03 [warning  ] Found entity which is not supported by Presidio entity=AGE
2024-12-01 05:39:03 [warning  ] Found unrecognized label, returning entity as is label=CURRENCYSYMBOL
2024-12-01 05:39:03 [warning  ] Found unrecognized label, returning entity as is label=CURRENCYSYMBOL
2024-12-01 05:39:03 [warning  ] Found unrecognized label, returning entity as is label=ACCOUNTNUMBER
2024-12-01 05:39:03 [debug    ] removing element type: US_SSN_RE, start: 139, end: 150, score: 0.75 from results list due to conflict
2024-12-01 05:39:03 [debug    ] removing element type: EMAIL_ADDRESS_RE, start: 162, end: 177, score: 0.75 from results list due to conflict
2024-12-01 05:39:03 [warning  ] Found sensitive data in the prompt and replaced it merged_results=[type: PERSON, start: 56, end

In [7]:
print(sanitized_prompt)

Please translate the following to Vietnamese: 

    "Hi [REDACTED_PERSON_1]! Please see below for your requested info

    full name: [REDACTED_PERSON_2]
    SSN: [REDACTED_US_SSN_1]
    email: [REDACTED_EMAIL_ADDRESS_1]
    birthday: 04/11/2003, so I am 21 years old
    What's your nationality? I am a Vietnamese citizen
    What's your credit score? 775 FICO
    What's the loan amount requested? $1,000,000
    What's the loan purpose? Invest $500,000 in Sustana Cooling Partners as working capital and cash out the rest for my hi-performing restaurant business (Beers & Bastards, LLC). 

    Please ping me the best terms you could offer. Thanks!"
    


In [8]:
# initialize a custome regex scanner func to address the PIIs missed above

def llm_guard_regex_scanner(starting_prompt, pattern):
    regex_scanner = Regex(
        patterns=[pattern],
        is_blocked=True,
        match_type=MatchType.SEARCH,
        redact=True
    )

    res = regex_scanner.scan(starting_prompt)[0]
    return res 

birthday_regex = r"\b\d{2}/\d{2}/\d{4}\b"
dollar_regex = r"\$\d[0-9,]*"
llc_regex = r"[a-zA-Z &]*, LLC"
partnership_regex = r"[A-Z][a-z ]* [A-Z][a-z ]*Partners"

patterns = [birthday_regex, dollar_regex, dollar_regex, llc_regex, partnership_regex]

In [9]:
# scan imperfect sanitized prompt again

prior_regex_prompt = sanitized_prompt

for pattern in patterns:
    result = llm_guard_regex_scanner(prior_regex_prompt, pattern)
    prior_regex_prompt = result

after_regex_prompt = prior_regex_prompt

2024-12-01 05:39:03 [warning  ] Pattern was detected in the text pattern=re.compile('\\b\\d{2}/\\d{2}/\\d{4}\\b')
2024-12-01 05:39:03 [warning  ] Pattern was detected in the text pattern=re.compile('\\$\\d[0-9,]*')
2024-12-01 05:39:03 [warning  ] Pattern was detected in the text pattern=re.compile('\\$\\d[0-9,]*')
2024-12-01 05:39:03 [warning  ] Pattern was detected in the text pattern=re.compile('[a-zA-Z &]*, LLC')
2024-12-01 05:39:03 [warning  ] Pattern was detected in the text pattern=re.compile('[A-Z][a-z ]* [A-Z][a-z ]*Partners')


In [10]:
print(after_regex_prompt)

Please translate the following to Vietnamese: 

    "Hi [REDACTED_PERSON_1]! Please see below for your requested info

    full name: [REDACTED_PERSON_2]
    SSN: [REDACTED_US_SSN_1]
    email: [REDACTED_EMAIL_ADDRESS_1]
    birthday: [REDACTED], so I am 21 years old
    What's your nationality? I am a Vietnamese citizen
    What's your credit score? 775 FICO
    What's the loan amount requested? [REDACTED]
    What's the loan purpose? Invest [REDACTED] in [REDACTED] as working capital and cash out the rest for my hi-performing restaurant business ([REDACTED]). 

    Please ping me the best terms you could offer. Thanks!"
    


In [11]:
# translate

res = client.chat.completions.create(
    model=GPT_MODEL,
    messages=[
        {'role': 'system', 'content': 'You are a helpful English-Vietnamese translation assistant.'},
        {'role': 'user', 'content': after_regex_prompt}
    ]
)
res_text = res.choices[0].message.content
print(res_text)

"Xin chào [REDACTED_PERSON_1]! Xin vui lòng xem thông tin bạn yêu cầu dưới đây:

    họ và tên: [REDACTED_PERSON_2]
    SSN: [REDACTED_US_SSN_1]
    email: [REDACTED_EMAIL_ADDRESS_1]
    ngày sinh: [REDACTED], vì vậy tôi 21 tuổi
    Quốc tịch của bạn là gì? Tôi là công dân Việt Nam
    Điểm tín dụng của bạn là bao nhiêu? 775 FICO
    Số tiền vay yêu cầu là bao nhiêu? [REDACTED]
    Mục đích vay vốn là gì? Đầu tư [REDACTED] vào [REDACTED] làm vốn lưu động và rút phần còn lại cho doanh nghiệp nhà hàng hiệu suất cao của tôi ([REDACTED]).

    Xin hãy gửi cho tôi các điều khoản tốt nhất bạn có thể cung cấp. Cảm ơn!"


In [12]:
# re-identify the results by adding the removed PIIs back

deanonymized_res_text = scan_output(
    output_scanners, after_regex_prompt, res_text
)
print(deanonymized_res_text[0])

2024-12-01 05:39:12 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_EMAIL_ADDRESS_1]
2024-12-01 05:39:12 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_US_SSN_1]
2024-12-01 05:39:12 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_PERSON_2]
2024-12-01 05:39:12 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_PERSON_1]
2024-12-01 05:39:12 [debug    ] Scanner completed              elapsed_time_seconds=0.013102 is_valid=True scanner=Deanonymize
2024-12-01 05:39:12 [info     ] Scanned output                 elapsed_time_seconds=0.018101 scores={'Deanonymize': 0.0}
"Xin chào Robert! Xin vui lòng xem thông tin bạn yêu cầu dưới đây:

    họ và tên: Phu Dang
    SSN: 123-45-1119
    email: pndang@ucsd.edu
    ngày sinh: [REDACTED], vì vậy tôi 21 tuổi
    Quốc tịch của bạn là gì? Tôi là công dân Việt Nam
    Điểm tín dụng của bạn là bao nhiêu? 775 FICO
    Số tiền vay yêu cầu là bao nhiêu? [REDACTED]
   

##### Notes on S1P1 (Protect AI)
- Optional or under-the-hood MS Presidio implementation
- Vault functionalities lost with custom regex (final output misses PIIs treated with custom regex)
- simply says "REDACTED", little human readability

final sanitized prompt:

    "Please translate the following to Vietnamese: 

    "Hi [REDACTED_PERSON_1]! Please see below for your requested info

    full name: [REDACTED_PERSON_2]
    SSN: [REDACTED_US_SSN_RE_1]
    email: [REDACTED_EMAIL_ADDRESS_1]
    birthday: [REDACTED], so I am 21 years old
    What's your nationality? I am a Vietnamese citizen
    What's your credit score? 775 FICO
    What's the loan amount requested? [REDACTED]
    What's the loan purpose? Invest [REDACTED] in [REDACTED] as working capital and cash out the rest for my hi-performing restaurant business ([REDACTED]). 

    Please ping me the best terms you could offer. Thanks!"

final output:

    "Xin chào Robert! Xin xem phía dưới để biết thông tin bạn đã yêu cầu

    Họ và tên: Phu Dang
    SSN: 123-45-6789
    Email: pndang@ucsd.edu
    Ngày sinh: [REDACTED], vì vậy tôi đã 21 tuổi
    Quốc tịch của bạn là gì? Tôi là công dân Việt Nam
    Điểm tín dụng của bạn là bao nhiêu? 775 FICO
    Số tiền vay yêu cầu là bao nhiêu? [REDACTED]
    Mục đích vay là gì? Đầu tư [REDACTED] vào [REDACTED] làm vốn lưu động và rút tiền mặt phần còn lại cho doanh nghiệp nhà hàng hiệu suất cao của tôi ([REDACTED]).

    Xin vui lòng gửi cho tôi những điều khoản tốt nhất bạn có thể cung cấp. Cảm ơn!"

______________

#### Guardrails AI's DetectPII + Microsoft Presidio up next!

Switch notebook to scen1prom1_guardrailsai.ipynb due to conflicting dependencies with Protect AI